In [1]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
from PIL import Image
from datetime import datetime
import matplotlib.pyplot as plt
from skimage import measure
import os, json, cv2, random, pathlib, shutil

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode
from detectron2.utils.visualizer import ColorMode

import helper as h

In [2]:
# Set Intput Arguments
seed = 0
dataset_path = "../../google-drive/"
dataset_name = "stomata200-mix"
dataset_postfix = '_val' # this can be set to ["", "_train", "_val", "_test"]
model_folder_name = "stomata200-mix_output_ep32000_2023_02_14_19_32_50"
test_threshold = 0.3 # this is the detection threshold
test_img_dir = "../../google-drive/stomata100-museum/images" # This can be any folder consisting of images.
test_img_num = 2 # set to None if you wish to run through the whole directory.

# check if test_img_num is valid
total_img_num = len([entry for entry in os.listdir(test_img_dir) if os.path.isfile(os.path.join(test_img_dir, entry))])
if test_img_num > total_img_num:
    print(f"test_img_num is too large. There are only {total_img_num} in {test_img_dir}")

CATEGORIES = ["Open", "close", "Unknown"]
INST_CATEGORIES = ["stomata"]
COLORS = {
    "Open": (0, 255, 0),  # Green
    "close": (255, 0, 0),  # Red
    "Unknown": (0, 0, 255)  # Blue
}

# Ensure you have 
DRAW_THICKNESS = {
    "stomata100": 2,
    "stomata100-museum": 2,
    "stomata200-mix": 2
}

if dataset_name not in DRAW_THICKNESS:
    print("dataset-Name is not defeind in DRAW_THICKNESS. Check if it is properly set.")

# Set Variables
random.seed(seed)
label_filename = os.path.join(dataset_path, dataset_name, "labels", f"labels{dataset_postfix}.json")
if os.path.isfile(label_filename) == False:
    print(f"Label file, {label_filename}, does not exist. Please check if the arguments above are correct and ensure that you have prepared the dataset with data_preparation.ipynb. Read READ.me for further information.")



## Register Dataset and Metadata

In [3]:
# We actually don't need to register DatasetCatelog here but let's ignore it for now.
for d in ["train", "val"]:
    catelog_name = "{}_{}".format(dataset_name, d)
    if catelog_name in DatasetCatalog:
        DatasetCatalog.remove(catelog_name)
    if catelog_name in MetadataCatalog:
        MetadataCatalog.remove(catelog_name)
    
    img_dir = os.path.join(dataset_path, catelog_name)
    DatasetCatalog.register(catelog_name, 
        lambda d=d: h.get_detectron2_dicts(os.path.join(dataset_path, dataset_name, d), label_filename))
    MetadataCatalog.get(catelog_name).set(thing_classes=INST_CATEGORIES)

stomata_metadata = MetadataCatalog.get("{}_train".format(dataset_name))

In [4]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = (f"{dataset_name}_train",)
cfg.DATASETS.TEST = (f"{dataset_name}_val", )
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 1000    # you may need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (stomata). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.
cfg.MODEL.DEVICE = "cuda"    # deploy model to device

cfg.OUTPUT_DIR = os.path.join("output",model_folder_name,"train_output")
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = test_threshold  # set a custom testing threshold


In [5]:
# set up save directory
save_dir = os.path.join(
    dataset_path, "inference_outputs", dataset_name, f"{model_folder_name}_inference", f"thres_{cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST}")
shutil.rmtree(save_dir, ignore_errors=True)
os.makedirs(save_dir)

predictor = DefaultPredictor(cfg)

# Get inference dicts
infer_dicts = h.get_inference_dicts(img_dir=test_img_dir)
random.shuffle(infer_dicts)

# Start inferencing
for idx, d in enumerate(infer_dicts[:test_img_num]):
    img_filename = d["file_name"] 
    im = cv2.imread(img_filename)
    outputs = predictor(im)  
    # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format

    # instance_mode=ColorMode.IMAGE_BW 
    # remove the colors of unsegmented pixels. This option is only available for segmentation models
    v = Visualizer(im[:, :, ::-1],
                    metadata=stomata_metadata,
                    scale=1,

        )
    
    # `.to.(cpu)` is to transform datatype from tensor to numpy.array.
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    
    # INSTANCE SEGMENTATION
    # Draw estimated INST SEG on images
    fig = plt.figure(figsize=(12, 8), dpi=600)
    fig.add_subplot(1, 2, 1)
    plt.imshow(out.get_image())
    plt.axis('off')
    plt.title("Pred: {}".format(os.path.basename(d["file_name"])))
    
    filename, file_extension = os.path.splitext(os.path.basename(d["file_name"]))
    # Set file extension to JPEG
    file_extension = ".jpg"
    
    # Save instance segmentation results
    save_filename = os.path.join(
        save_dir, 
        "{}_thres_{}_inst_seg{}".format(filename, cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST, file_extension))
    cv2.imwrite(save_filename, cv2.cvtColor(out.get_image(), cv2.COLOR_BGR2RGB))
    
    # ROTATED BBOX
    # Draw estimated ROTATED BBOX on images
    pred_masks = outputs['instances'].pred_masks
    pred_categories = outputs['instances'].pred_classes
    pred_scores = outputs['instances'].scores
    polygons, categories = [], []
    for mask, category, score in zip(pred_masks, pred_categories, pred_scores):
        po = h.binary_mask_to_polygon(mask.cpu())
        polygons += po
        categories += ["{} {:.0%}".format(INST_CATEGORIES[category], score)] * len(po)
    
    # Draw fitted ROTATED BBOX
    fitted_rbboxs = h.fit_polygens_to_rotated_bboxes(polygons)

    img_draw = h.draw_rotated_bboxes(img_filename, fitted_rbboxs, categories, thickness=DRAW_THICKNESS[dataset_name], color=None)
    fig.add_subplot(1, 2, 2)
    plt.imshow(img_draw)
    plt.axis('off')
    plt.title("Pred_rotated_bboxes: {}".format(os.path.basename(d["file_name"])))
    
    # Save ROTATED_BBOX result to file
    save_filename = os.path.join(
        save_dir, 
        "{}_thres_{}_rotated_bbox{}".format(filename, cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST, file_extension))
    cv2.imwrite(save_filename, cv2.cvtColor(img_draw, cv2.COLOR_BGR2RGB))
    
    # OUTPUT RESULTS FOR BOTH TASKS
    # Save both INST SEG and ROTATED BBOX results to file.
    save_filename = os.path.join(
        save_dir, 
        "benchmark_{}_thres_{}{}".format(filename, cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST, file_extension))
    fig.savefig(save_filename, bbox_inches='tight')

[02/17 15:16:13 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from output/stomata200-mix_output_ep32000_2023_02_14_19_32_50/train_output/model_final.pth ...


/home/pychen/anaconda3/envs/abrc/lib/python3.9/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
